In [2]:
import numpy as np 
import pandas as pd
import os
import glob
import re
import sys

import plotnine
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import animation, rc
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
import folium
from folium import plugins

In [3]:
import json

from branca.element import CssLink, Figure, JavascriptLink, MacroElement
from jinja2 import Template

In [4]:
files = glob.glob('dataset/*.csv')
data = []
for i in files :
    i = re.sub("dataset/", "", i)
    data.append(re.sub(".csv","",i))
for i in range(len(files)) :
    globals()[data[i]] = pd.read_csv(files[i])
data

['TimeAge',
 'SearchTrend',
 'TimeProvince',
 'Weather',
 'PatientRoute',
 'PatientInfo',
 'Region',
 'TimeGender',
 'Case',
 'Time']

In [5]:
route_geo = gpd.GeoDataFrame(PatientRoute,geometry=gpd.points_from_xy(PatientRoute.longitude,PatientRoute.latitude))

## 연습삼아 제일 많이 돌아다녔던 사람의 경로만 그려보자

In [34]:
inssa = route_geo[route_geo["patient_id"] == 1000000013].reset_index(drop=True)

점으로 된 경로를 line으로 이어준다

In [35]:
inssa_lines = [ {
                  "coordinates": [
                    [inssa["longitude"][i] , inssa["latitude"][i] ],
                    [inssa["longitude"][i+1] , inssa["latitude"][i+1],]
                  ],
                  "dates": [
                      inssa["date"][i],
                      inssa["date"][i+1]
                  ]
                     }
              for i in range(len(inssa)-1) ] 

In [37]:
m = folium.Map(
  location=[inssa.latitude[0],inssa.longitude[0]],
  tiles='Stamen Toner',
  zoom_start=10
)

features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'LineString',
            'coordinates': line['coordinates'],
        },
        'properties': {
            'times': line['dates'],
        }
    }
    for line in inssa_lines
]

plugins.TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features,
}, period='PT3H', add_last_point=True, ).add_to(m)
m.save("inssa_route.html")
m

## 전체 환자들을 올려보자

In [94]:
linerange = route_geo[route_geo['patient_id'] == route_geo['patient_id'].shift(1)].index
linerange = linerange.drop(linerange[-1])

In [131]:
lines = [ {
                "coordinates": [
                 [route_geo["longitude"][i] , route_geo["latitude"][i] ],
                 [route_geo["longitude"][i+1] , route_geo["latitude"][i+1],]
                   ],
                "dates": [
                 route_geo["date"][i],
                 route_geo["date"][i+1]
                   ],
                     } for i in linerange ] 
features = [
     {
         'type': 'Feature',
         'geometry': {
            'type': 'LineString',
            'coordinates': line['coordinates'],
                },
         'properties': {
            'times': line['dates'],
            'icon' : 'circle',
            'iconstyle':{
                    'fillOpacity': 0.8,
                    'stroke': 'true',
                    'radius': 3
                }
                }
        }
        for line in lines
    ]

In [132]:
m = folium.Map(
  location=[36.5053542,127.7043419],
  zoom_start=8
)
plugins.TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features,
}, period='PT6H', add_last_point=True).add_to(m)

In [133]:
m.save('without_color.html')
m